In [2]:
# Import necessary libraries
from enum import auto
import json
import joblib
import os
import streamlit as st
from itertools import cycle
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os
import urllib
from skimage import io
import pdb
#*** added
from PIL import Image
import webcolors
#***
import seaborn as sns
import pandas as pd
import base64
from art_api import utils
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Predict colour

1. Obtain image from user
2. Resize image and store in dir_sm as img_sm
4. Read img_sm from dir_sm
5. Get colours from image_sm
6. Use yellowbrick to generate optimal K
7. Optimal K used to generate df of up to 7 optimal colours per image
8. Display to user
9. Save to CSV